In [ ]:
from flask import Flask, request, jsonify
import requests
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

app = Flask(__name__)

# Load datasets
df1 = pd.read_csv("Crop_recommendation.csv")
df2 = pd.read_csv("CRUSP.csv")
df_combined = pd.concat([df1, df2], ignore_index=True)

# Preprocessing
label_encoder = LabelEncoder()
df_combined['label'] = label_encoder.fit_transform(df_combined['label'])
numerical_cols = [col for col in df_combined.columns if col != 'label']
scaler = StandardScaler()
df_combined[numerical_cols] = scaler.fit_transform(df_combined[numerical_cols])

# Train the model
X = df_combined[numerical_cols]
y = df_combined['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=300, max_depth=25, random_state=42)
model.fit(X_train, y_train)

# API Keys
API_KEY = "ed3f7c277617620ec97ffe73b5362ce7"

# Functions to fetch weather and soil data
def get_coordinates(location):
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={location}&limit=1&appid={API_KEY}"
    response = requests.get(url)
    data = response.json()
    if data:
        return data[0]['lat'], data[0]['lon']
    return None, None

def get_weather_data(lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    if 'main' in data:
        return data['main'].get('temp', None), data['main'].get('humidity', None), data.get('rain', {}).get('1h', 0)
    return None, None, None

@app.route('/predict', methods=['POST'])
def predict():
    location = request.form.get("location")  # Get location from form
    if not location:
        return jsonify({"error": "No location provided"}), 400

    lat, lon = get_coordinates(location)
    if lat is None:
        return jsonify({"error": "Invalid location"}), 400

    temperature, humidity, rainfall = get_weather_data(lat, lon)

    input_data = {
        'temperature': temperature,
        'humidity': humidity,
        'rainfall': rainfall
    }

    input_df = pd.DataFrame([input_data], columns=numerical_cols)
    input_df[numerical_cols] = scaler.transform(input_df[numerical_cols])

    probabilities = model.predict_proba(input_df)[0]
    crop_labels = label_encoder.inverse_transform(model.classes_)
    top_indices = probabilities.argsort()[-10:][::-1]
    
    top_crops = [{"crop": crop_labels[i], "confidence": round(probabilities[i] * 100, 2)} for i in top_indices]

    return jsonify({"location": location, "recommended_crops": top_crops})

if __name__ == '__main__':
    app.run(debug=True)


Available Crops: Barley, coconut, Red Pepper, Sorghum, orange, Bean, mothbeans, mungbean, pomegranate, Potato, Teff, Pea, Dagussa, Niger seed, jute, Fallow, Wheat, lentil, pigeonpeas, coffee, rice, chickpea, grapes, apple, blackgram, kidneybeans, muskmelon, maize, watermelon, mango, banana, papaya, Maize, cotton
RandomForest Model Accuracy: 67.87%
Enter your location (city, village, or specific area): Kerala

Top 10 Recommended Crops for Kerala:
1. Teff (Confidence: 27.27%) - Best suited due to temperature: 16.13°C, humidity: 14%, rainfall: 0mm, and soil properties.
2. chickpea (Confidence: 12.67%) - Best suited due to temperature: 16.13°C, humidity: 14%, rainfall: 0mm, and soil properties.
3. Wheat (Confidence: 10.42%) - Best suited due to temperature: 16.13°C, humidity: 14%, rainfall: 0mm, and soil properties.
4. mothbeans (Confidence: 8.67%) - Best suited due to temperature: 16.13°C, humidity: 14%, rainfall: 0mm, and soil properties.
5. muskmelon (Confidence: 7.67%) - Best suited du